# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм вида: `<дата> - <заголовок> - <ссылка>`

In [51]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from datetime import timedelta
from pymystem3 import Mystem
import locale    # Для парсинга даты, которая спрятана в 'req.headers'
locale.setlocale(locale.LC_ALL, 'rus_RUS')
import nltk
from nltk.corpus import stopwords
from string import punctuation

import pymorphy2 as pym2
morph = pym2.MorphAnalyzer()


mystem = Mystem()

russian_stopwords = set(stopwords.words("russian"))   # Поиск по нмножеству быстрее
all_punctuation = set(punctuation)                    # Поиск по нмножеству быстрее

In [2]:
KEYWORDS = ['python', 'парсинг', 'big', 'data', 'ios', 'cloud', 'android', 'java', 'linux']

In [52]:
# --- Функция разбивки русского текста на леммы. Выдает множество лемм или стемм

a = 'Мой опыт с экзаменами GCP: Associate, Architect, Network'

def lematizator(sentence, lemm = True , pymorphy = False):
    """ 
    Функция разбивки русского текста на леммы или стеммы. 
    Выдает множество лемм  (если lemm = True) 
    или множество стемм (pymorphy = True)
    """
    words = set( nltk.word_tokenize(sentence) )

    without_stop_words = [word.lower() for word in words if (word.lower() not in russian_stopwords) and (word not in all_punctuation) ]

    if lemm:
        pymorphy = False
        without_stop_words_and_lemmatized = [ mystem.lemmatize(word)[0] for word in without_stop_words  ]
        return set(without_stop_words_and_lemmatized)
    elif pymorphy:
        without_stop_words_and_morphytized = [ morph.parse(word)[0].normal_form for word in without_stop_words  ]
        return set(without_stop_words_and_morphytized)
    else: 
        return set(without_stop_words)

lematizator(a,lemm = False , pymorphy = True )

{'architect', 'associate', 'gcp', 'network', 'опыт', 'экзамен'}

In [4]:
# --- Функция вернет время публикации

a = 'сегодня в 16:38'

def get_post_time(today, yesterday, post_time):
    """ 
    Функция вернет время публикации  
    """
    if post_time.split()[0] == 'сегодня':
        post_time = today.strftime('%Y_%m_%d')  
    elif post_time.split()[0] == 'вчера':
        post_time = yesterday.strftime('%Y_%m_%d')
    else:                    
        dd = mystem.lemmatize(post_time)
        dd = ''.join(dd)
        locale.setlocale(locale.LC_ALL, 'rus_RUS')
        post_time_in_datetime = datetime.strptime( dd , '%d %B %Y в %H:%M  \n')
        post_time = post_time_in_datetime.strftime('%Y_%m_%d')
        
    return post_time

#get_post_time(a)

In [59]:
#word = 'ghj'

publication_le = {'протокол', 'работа', 'работа', 'пример', 'ftp'}

KEYWORDS_in_title = [word for word in KEYWORDS if word in publication_le]
KEYWORDS_in_title

[]

In [65]:
not (not KEYWORDS_in_title)

True

In [66]:
# --- Функция main

def make_requests( pages = 2, debug = True):
    
    A = pd.DataFrame(columns = ['дата' , 'заголовок', 'ссылка'] ) 
    
    headers = {
        "Host" : "habr.com",
        "Accept-Encoding" : "gzip, deflate, br",
        "Cache-Control": "max-age=0",
        "Connection" : "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Accept" : "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
        "Accept-Language" : "en-US,en;q=0.5",
        "User-Agent" : "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:85.0) Gecko/20100101 Firefox/85.0"
                }
    
    for i in range(pages):
        URL = f'https://habr.com/ru/page{i}/'
        req1 = requests.get(URL, headers = headers)
        
        locale.setlocale(locale.LC_ALL, 'en_US')
        today = datetime.strptime( req1.headers['Date'] , '%a, %d %b %Y %H:%M:%S %Z')
        yesterday = today + timedelta(days = -1)
        
        time.sleep(0.3)
        
        soup = BeautifulSoup(req1.text, 'html.parser')
        big_post = soup.find_all('article', class_='post post_preview')    
        
        #------ Разбиваем на публикации 
        
        for element in big_post:         
            if element.find('a', class_='post__title_link'):   # Публикация найдена
 
                publication_title = element.find('a', class_='post__title_link').text
                if debug: print(publication_title)

                #------ Бъем публикацию на леммы (lematizator) и сохраняем в списке 'publication'         
                
                publication_lemmatized = lematizator(publication_title, lemm = False , pymorphy = True )
                if debug: print(publication_lemmatized)
                    
                KEYWORDS_in_title = [word for word in KEYWORDS if word in publication_lemmatized]

                if not (not KEYWORDS_in_title):    # Feel the Force, Luke! KEYWORD is Here!
                        post_time = element.find('span', class_='post__time').text.lower()
                        post_time = get_post_time(today, yesterday, post_time)
                        link = element.find('a', class_='post__title_link').get('href')
                         
                        if debug:
                            print(f'Совпадение найдено: {KEYWORDS_in_title}') 
                            print(f'время публикации: {post_time}')
                            print(f'Название публикации: {publication_title}')
                            print(f'Ссылка на публикацию: {link}')
                            B = pd.DataFrame( [ [post_time, publication_title, link, KEYWORDS_in_title] ], columns = ['дата','заголовок', 'ссылка', 'ключевое_слово'] )
                        else:
        #-------------- Добавляем в список полученную публикацию в формате '<дата> - <заголовок> - <ссылка>' ----#        
                            B = pd.DataFrame( [ [post_time, publication_title, link] ], columns = ['дата','заголовок', 'ссылка'] )        
                        
                        A = pd.concat( [ A , B ] )
                    
            
    return A


output_report = make_requests(pages=2)
output_report

Xiaomi Gateway MIEU01 как универсальный контроллер умного дома
{'дом', 'универсальный', 'умный', 'контроллер', 'gateway', 'mieu01', 'xiaomi'}
HR-тренды 2020-2021: лидерство, wellbeing и антихрупкость
{'2020-2021', 'антихрупкость', 'wellbeing', 'hr-тренд', 'лидерство'}
Кто использует магнитную плёнку и почему за ней будущее
{'почему', 'магнитный', 'будущее', 'плёнка', 'использовать'}
Создание исполняемого файла ELF вручную
{'создание', 'файл', 'исполняемый', 'elf', 'вручную'}
Как мы раскрыли 24-летний баг в ядре Linux
{'24-летний', 'ядро', 'баг', 'linux', 'раскрыть'}
Совпадение найдено: ['linux']
время публикации: 2021_02_22
Название публикации: Как мы раскрыли 24-летний баг в ядре Linux
Ссылка на публикацию: https://habr.com/ru/company/ruvds/blog/543134/
Извилистая геометрия путешествий вокруг света
{'вокруг', 'геометрия', 'свет', 'путешествие', 'извилистый'}
Как скоро «цифровые люди» захватят приложения для знакомств?
{'человек', 'знакомство', '»', 'приложение', 'скоро', 'захватить', 

,дата,заголовок,ссылка,ключевое_слово
0,2021_02_22,Как мы раскрыли 24-летний баг в ядре Linux,https://habr.com/ru/company/ruvds/blog/543134/,[linux]
0,2021_02_22,Рейтинг языков программирования 2021: доля Pyt...,https://habr.com/ru/post/543346/,"[python, java]"


### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.  

Итоговый датафрейм формировать со столбцами: `<дата> - <заголовок> - <ссылка> - <текст_статьи>`


In [49]:
def check_function(pages = 1 , debug = True):
    for i in range(1, pages + 1,1):
        URL = f'https://habr.com/ru/page{i}/'
        
        req = requests.get(URL)
        A = pd.DataFrame(columns = ['дата' , 'заголовок', 'ссылка', 'текст_статьи'] )

        time.sleep(0.3)

        locale.setlocale(locale.LC_ALL, 'en_US')
        today = datetime.strptime( req.headers['Date'] , '%a, %d %b %Y %H:%M:%S %Z')
        yesterday = today + timedelta(days = -1)

        soup = BeautifulSoup(req.text, 'html.parser')
        links = [element.get('href') for element in soup.find_all('a', class_='post__title_link') ]
        
        for link in links:
            req = requests.get(link)
            soup = BeautifulSoup(req.text, 'html.parser')

            time.sleep(0.5)

            publicaton_title = soup.find('span', class_='post__title-text').text
            if debug: print(publicaton_title)
            publicaton_text = soup.find('div', class_='post__body post__body_full').text.replace('\n','').replace('\r','')

            lemmatized_publicaton_title = lematizator(publicaton_title, lemm = False , pymorphy = True )
            KEYWORDS_in_title = [word for word in lemmatized_publicaton_title if word in set(KEYWORDS)]

            if (not KEYWORDS_in_title):
                lemmatized_publicaton_text = lematizator(publicaton_text, lemm = False , pymorphy = True )
                KEYWORDS_in_text = [word for word in lemmatized_publicaton_text if word in set(KEYWORDS)]
                if (not KEYWORDS_in_text): continue
            else: 
                KEYWORDS_in_text = []

         # Нашли ключевое слово
            post_time = soup.find('span', class_='post__time').text.lower()
            post_time = get_post_time(today, yesterday, post_time)

            if debug:
                print(f'время публикации: {post_time}')
                print(f'Название публикации: {publicaton_title}')
                print(f'Ссылка на публикацию: {link}')
                #print(f'Текст публикации: {publicaton_text}') 
                found_keyword = ''.join(KEYWORDS_in_title +[" "]+ KEYWORDS_in_text)
                print(f'Ключевое слово: {found_keyword}')
                print()
                B = pd.DataFrame( [ [post_time, publicaton_title, link, publicaton_text, found_keyword] ], columns = ['дата','заголовок', 'ссылка', 'текст_статьи', 'ключевое слово'] )        
            else:
                B = pd.DataFrame( [ [post_time, publicaton_title, link, publicaton_text] ], columns = ['дата','заголовок', 'ссылка', 'текст_статьи'] )        

            A = pd.concat( [ A , B ] ) 

    return A

check_function(1)

Дайджест интересных материалов для мобильного разработчика #382 (15 — 21 февраля)
время публикации: 2021_02_21
Название публикации: Дайджест интересных материалов для мобильного разработчика #382 (15 — 21 февраля)
Ссылка на публикацию: https://habr.com/ru/company/productivity_inside/blog/543528/
Ключевое слово:  android
Заметки о Unix: история Unix до readline
Assembler Editor Plus: Добавление нового микроконтроллера
Эффективная конструкция агрегатов. Моделирование одиночного агрегата
HackTheBox. Прохождение Feline. RCE через сереализацию в Java и LPE через докер сокеты
время публикации: 2021_02_21
Название публикации: HackTheBox. Прохождение Feline. RCE через сереализацию в Java и LPE через докер сокеты
Ссылка на публикацию: https://habr.com/ru/post/542612/
Ключевое слово: java 
Linux, suspend to RAM и ИБП
время публикации: 2021_02_21
Название публикации: Linux, suspend to RAM и ИБП
Ссылка на публикацию: https://habr.com/ru/post/543514/
Ключевое слово: linux 
Личный кабинет адаптации 

,дата,заголовок,ссылка,текст_статьи,ключевое слово
0,2021_02_21,Дайджест интересных материалов для мобильного ...,https://habr.com/ru/company/productivity_insid...,"В этом выпуске цвета Swift, переиспользуемый ч...",android
0,2021_02_21,HackTheBox. Прохождение Feline. RCE через сере...,https://habr.com/ru/post/542612/,Продолжаю публикацию решений отправленных на д...,java
0,2021_02_21,"Linux, suspend to RAM и ИБП",https://habr.com/ru/post/543514/,"В случае периодических, но достаточно кратковр...",linux
0,2021_02_20,Работа с FTP протоколом в Android. Пример,https://habr.com/ru/post/543502/,Всем привет! Это будет очень маленькая статья....,android
0,2021_02_20,Из-за пандемии потребление интернет-трафика пр...,https://habr.com/ru/company/selectel/blog/543482/,Компания TeleGeography проанализировала показа...,cloud
0,2021_02_20,Делаем систему контроля и управления доступом ...,https://habr.com/ru/post/543446/,ВведениеПримерно год назад я начал готовиться ...,data
0,2021_02_20,Поддержка токенов PKCS#11 с ГОСТ-криптографией...,https://habr.com/ru/post/542182/,Поддержка криптографических токенов PKCS#11 с ...,python
0,2021_02_20,"Темы, стили и атрибуты",https://habr.com/ru/post/543460/,В Android существуют стили и темы которые позв...,android


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: `<почта> - <дата утечки> - <источник утечки> - <описание утечки>`  

**Подсказка**: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [199]:
import json

EMAIL = ['kate1991@yandex.ru', 'yyy@y.com']

def check_email_for_leak(EMAIL):
    
    A = pd.DataFrame(columns = ['почта' , 'дата утечки', 'источник утечки', 'описание утечки'] )

    URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

    headers = {

        'User-Agent ': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:85.0) Gecko/20100101 Firefox/85.0',
        'Vaar-Header-App-Product' : 'hackcheck-web-avast',
        'Vaar-Header-App-Product-Name' : 'hackcheck-web-avast',
        'Vaar-Header-Build-Version' : '{__VERSION__}',
        'Vaar-Version' : '0',
        'Content-Type' : 'application/json;charset=utf-8',
        'Content-Length' : '41',
        'Origin' : 'https://www.avast.com',
        'Referer' : 'https://www.avast.com/hackcheck/'
        } 
    for email in EMAIL:
        print(email)
        params = { 'emailAddresses' : [ email ]  }

        req = requests.post(URL , data = json.dumps(params) , headers = headers)

        time.sleep(5)

        if not req.json():continue

        C = req.json()
        C['breaches'].keys()
        for breach_id in C['breaches'].keys():
            breach_source = C['breaches'][breach_id]['site']
            breach_description = req.json()['breaches'][breach_id]['description']
            breach_date = req.json()['breaches'][breach_id]['publishDate']
            breach_date_in_datetime = datetime.strptime( breach_date , '%Y-%m-%dT%H:%M:%SZ')
            breach_date = breach_date_in_datetime.strftime('%Y_%m_%d')
            B = pd.DataFrame( [ [email, breach_date, breach_source, breach_description ] ], columns = ['почта' , 'дата утечки', 'источник утечки', 'описание утечки'] )        
            A = pd.concat( [ A , B ] )
    A = A.reset_index()
    return A

leaked_emails = check_email_for_leak(EMAIL)
leaked_emails

kate1991@yandex.ru
yyy@y.com


,index,почта,дата утечки,источник утечки,описание утечки
0,0,kate1991@yandex.ru,2016_10_29,vk.com,Popular Russian social networking platform VKo...
1,0,yyy@y.com,2020_01_03,azcentral.com,"At an unconfirmed date, online Arizona newspap..."
2,0,yyy@y.com,2020_05_28,wishbone.io,"In January 2020, the online poll website Wishb..."
3,0,yyy@y.com,2017_11_04,myheritage.com,"In October 2017, a customer database belonging..."
4,0,yyy@y.com,2021_02_11,forums.vkmonline.com,"At an unconfirmed date, the Russian-language m..."
5,0,yyy@y.com,2019_06_13,canva.com,"In May 2019, graphic-design site Canva's datab..."
6,0,yyy@y.com,2016_10_24,dropbox.com,Cloud storage company Dropbox suffered a major...
7,0,yyy@y.com,2016_10_21,linkedin.com,"In 2012, online professional networking platfo..."
8,0,yyy@y.com,2017_03_01,rayli.com.cn,"On an unconfirmed date, Chinese gossip site Ra..."
9,0,yyy@y.com,2019_10_17,zynga.com,"In September 2019, the game developer Zynga wa..."


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)  
```
GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ  
```

В итоге должен формироваться датафрейм со столбцами: `<дата поста> - <текст поста>`

In [ ]:
у меня нет профиля Вконтакте

#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
